In [ ]:
import pandas as pd
import json
import time
import random
import copy

with open("test_data.json", "r") as f:
    test_data = json.loads(f.read())
df = pd.DataFrame(test_data)

def random_date():
    start = time.mktime(time.strptime("2015-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    end = time.mktime(time.strptime("2026-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    random_date = start + random.random() * (end - start)
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(random_date))

random_dates = [random_date() for x in range(len(df["index"]))]
df["timestamp"] = random_dates

df.dropna()
df2 = copy.copy(df)
df3 = pd.concat([df, df2])

with open("data2.csv", "w") as f:
    df3.to_csv(f, index=False)